## Lab 2: Logic

- Full Name: Nguyễn Phúc Khang
- Student ID: 22127180

<font color='red'>**Attention:**</font>

You can discuss ideas with classmates as well as finding information from the internet, books, etc.; but *this homework must be yours*.

**How to submit your homework**

Before submitting, re-run the notebook (`Kernel` ->` Restart & Run All`).

Create folder named `Student_ID` (for example, if your Student ID is 1234567, you will name the folder as `1234567`) with the following structure:

    - File `Lab2.ipynb` that you have completed. 
    - Folder `models` and file `logic.py`to run the notebook above.
    - Folder `Exercise_4` that include your works on exercise 4:
        - `Source`: Source code.
        - `Report.pdf`: Report file.

Compress `Student_ID` folder with <font color='red'>**.zip**</font> format.

<font color=red>Please, follow exactly all the requirements and instructions above. </font>



## 0. How to do your homework

For exercises 1-3, you will work directly on this notebook. The sentence ```# YOUR CODE HERE``` will indicate the parts you need to do. Mathematical symbols must be presented with code as in the following symbol list: 

<center>
    
|  | Math symbols | Code | 
|:---|:----:|---|
| Constant symbol | hcmus | ```Constant('hcmus')``` (lowercase) | 
| Variable symbol | $x$ | ```Variable('$x')``` (lowercase) | 
| Atomic formula | Rain, LocatedIn(hcmus, $x$) | ```Atom('Rain')```, ```Atom('LocatedIn', 'hcmus', '$x')``` <br> (predicate in uppercase, argument in lowercase) |
| Negation | $\neg$ Rain | ```Not(Atom('Rain'))```|
| Conjunction | Rain $\land$ Snow | ```And(Atom('Rain'), Atom('Snow'))```|
| Disjunction | Rain $\lor$ Snow | ```Or(Atom('Rain'), Atom('Snow'))```|
| Implication | Rain $\Rightarrow$ Wet | ```Implies(Atom('Rain'), Atom('Wet'))```|
| Equivalence | Rain $\Leftrightarrow$ Wet | ```Equiv(Atom('Rain'), Atom('Wet'))```|
| Existential quantification | $\exists x$ LocatedIn(hcmus,$x$) | ```Exists('$x', Atom('LocatedIn', 'hcmus', '$x'))```|
| Universal quantification | $\forall x$ MadeOfAtoms($x$) | ```Forall('$x', Atom('MadeOfAtoms', '$x'))```|    
| Conjunction with multiple arguments | A $\land$ B $\land$ C | ```AndList([Atom('A'), Atom('B'), Atom('C')]) ```|
| Disjunction with multiple arguments | A $\lor$ B $\lor$ C | ```OrList([Atom('A'), Atom('B'), Atom('C')]) ```|  
    
</center>

In [1]:
import os
import gzip
import pickle
import random

from logic import *
from typing import Tuple, List

def checkFormula(name, predForm, preconditionForm=None):
    filename = os.path.join('models', name + '.pklz')
    objects, targetModels = pickle.load(gzip.open(filename)) 
    
    # If preconditionion exists, change the formula to
    preconditionPredForm = And(preconditionForm, predForm) if preconditionForm else predForm
    predModels = performModelChecking([preconditionPredForm], findAll=True, objects=objects)
    
    def hashkey(model): return tuple(sorted(str(atom) for atom in model))
    targetModelSet = set(hashkey(model) for model in targetModels)
    predModelSet = set(hashkey(model) for model in predModels)
    for model in targetModels:
        if hashkey(model) not in predModelSet:
            print("Your formula (%s) says the following model is FALSE, but it should be TRUE:" % predForm)
            printModel(model)
            return
    for model in predModels:
        if hashkey(model) not in targetModelSet:
            print("Your formula (%s) says the following model is TRUE, but it should be FALSE:" % predForm)
            printModel(model)
            return
        
    print(f"You matched the {len(targetModels)} models")
    print(f"Example model: {rstr(random.choice(targetModels))}")
    return True

## 1. Prepositional logic (0.75 pts)

For example: "If it 's rain, the road will be wet":

```
def rainWet():
    Rain = Atom('Rain') 
    Wet = Atom('Wet')
    return Implies(Rain, Wet)
```


**1a) If it's summer in California, it will not rain**

In [2]:
def formula1a() -> Formula:
    # Predicates to use:
    Summer = Atom('Summer')               # whether it's summer
    California = Atom('California')       # whether we're in California
    Rain = Atom('Rain')                   # whether it's raining
    
    # YOUR CODE HERE
    return Implies(And(Summer, California), Not(Rain))
    

In [3]:
assert checkFormula('1a', formula1a()) == True

You matched the 7 models
Example model: {'Summer'}


**1b) The road is wet if and only if the weather is rainy or the sprinkler is opened**

In [4]:
def formula1b() -> Formula:
    # Predicates to use:
    Rain = Atom('Rain')              # whether it is raining
    Wet = Atom('Wet')                # whether it it wet
    Sprinklers = Atom('Sprinklers')  # whether the sprinklers are on
    
    # YOUR CODE HERE
    return Equiv(Wet, Or(Rain, Sprinklers))
    

In [5]:
assert checkFormula('1b', formula1b()) == True

You matched the 4 models
Example model: {'Rain', 'Wet'}


**1c) Day or night**

In [6]:
def formula1c() -> Formula:
    # Predicates to use:
    Day = Atom('Day')     # whether it's day
    Night = Atom('Night') # whether it's night
    
    # YOUR CODE HERE
    return And(Or(Day, Night), Not(And(Day, Night)))
    

In [7]:
assert checkFormula('1c', formula1c()) == True

You matched the 2 models
Example model: {'Night'}


## 2. First order logic (2 pts)

Formulate statement from natural languages to prepositional logic.

For example: "There is a lighting source that shines."

```
def lightShines():
    def Light(x): return Atom('Light', x)
    def Shines(x): return Atom('Shines', x)
    return Exists('$x', And(Light('$x'), Shines('$x')))
```

**2a) Everybody has mother**

In [8]:
def formula2a() -> Formula:
    # Predicates to use:
    def Person(x): return Atom('Person', x)        # whether x is a person
    def Mother(x, y): return Atom('Mother', x, y)  # whether x's mother is y

    # Note: You do NOT have to enforce that the mother is a "person"
    # YOUR CODE HERE
    return Forall('$x', Implies(Person('$x'), Exists('$y', Mother('$x', '$y'))))

In [9]:
formula2a_precondition = AntiReflexive('Mother')
assert checkFormula('2a', formula2a(), formula2a_precondition) == True

You matched the 343 models
Example model: {'Person(o1)', 'Mother(o3,o2)', 'Person(o3)', 'Mother(o1,o2)', 'Mother(o2,o3)'}


**2b) At least one person does not have any child**

In [10]:
def formula2b() -> Formula:
    # Predicates to use:
    def Person(x): return Atom('Person', x)        # whether x is a person
    def Child(x, y): return Atom('Child', x, y)    # whether x has a child y

    # Note: You do NOT have to enforce that the child is a "person"
    # YOUR CODE HERE
    return Exists('$x', And(Person('$x'), Not(Exists('$y', Child('$x', '$y')))))
    

In [11]:
formula2b_precondition = AntiReflexive('Child')
assert checkFormula('2b', formula2b(), formula2b_precondition) == True

You matched the 169 models
Example model: {'Person(o1)', 'Child(o1,o2)', 'Child(o2,o3)', 'Person(o3)'}


**2c) Create definition for Daughter by Female and Child**

Examples:

```
# Defnition of Parent by Child
def parentChild():
    def Parent(x, y): return Atom('Parent', x, y)  # whether x has a parent y
    def Child(x, y): return Atom('Child', x, y)    # whether x has a child y
    return Forall('$x', Forall('$y', Equiv(Parent('$x', '$y'), Child('$y', '$x'))))
```

In [12]:
def formula2c() -> Formula:
    # Predicates to use:
    def Female(x): return Atom('Female', x)            # whether x is female
    def Child(x, y): return Atom('Child', x, y)        # whether x has a child y
    def Daughter(x, y): return Atom('Daughter', x, y)  # whether x has a daughter y
    
    # YOUR CODE HERE
    return Forall('$x', Forall('$y', Equiv(Daughter('$x', '$y'), And(Female('$y'), Child('$x', '$y')))))

In [13]:
formula2c_precondition = AntiReflexive('Child')
assert checkFormula('2c', formula2c(), formula2c_precondition) == True

You matched the 512 models
Example model: {'Daughter(o3,o1)', 'Child(o3,o1)', 'Child(o2,o3)', 'Female(o1)', 'Child(o1,o3)', 'Child(o3,o2)'}


**2d) Create definition for Grandmother by Female and Parent**

In [14]:
# Note: It is ok for a person to be her own parent
def formula2d() -> Formula:
    # Predicates to use:
    def Female(x): return Atom('Female', x)                  # whether x is female
    def Parent(x, y): return Atom('Parent', x, y)            # whether x has a parent y
    def Grandmother(x, y): return Atom('Grandmother', x, y)  # whether x has a grandmother y

    # YOUR CODE HERE
    return Forall('$x', Forall('$y', Equiv(
        Grandmother('$x', '$y'), 
        And(Female('$y'), Exists('$z', And(Parent('$x', '$z'), Parent('$z', '$y'))))
    )))

In [15]:
formula2d_precondition = AntiReflexive('Parent')
assert checkFormula('2d', formula2d(), formula2d_precondition) == True

You matched the 512 models
Example model: {'Grandmother(o2,o1)', 'Parent(o1,o3)', 'Female(o2)', 'Female(o1)', 'Grandmother(o3,o3)', 'Parent(o3,o2)', 'Grandmother(o2,o2)', 'Female(o3)', 'Parent(o3,o1)', 'Grandmother(o1,o2)', 'Grandmother(o1,o1)', 'Parent(o2,o3)'}


## 3. Who is the liar? (2.25đ)

Using prepositional method for the following problem:

A Facebook employee make a serious mistake that crashed the server, people are discussing to find out the one that caused this problem. After a while, four suspects are identified.

We have the following information:
- John: "I am not the guy that make the server down!"
- Susan: "It should be Nicole."
- Mark: "No, it was Susan!"
- Nicole: "Susan is lying, you should not trust her."
- There is **only 1 person** told the truth.
- There is **only 1 person** crashed the server.

Finish the ```liar()``` function with 6 formulas for finding the liar!

In [16]:
# Exercise 3: Liar puzzle

# Facts:
# 0. John: "It wasn't me!"
# 1. Susan: "It was Nicole!"
# 2. Mark: "No, it was Susan!"
# 3. Nicole: "Susan's a liar."
# 4. Exactly one person is telling the truth.
# 5. Exactly one person crashed the server.
# Query: Who did it?

# This function returns a list of 6 formulas corresponding to each of the
# above facts. Be sure your formulas are exactly in the order specified. 


# Hint: For fact 4 & 5, you might want to use the Equals predicate, defined as:
# def Equals(x, y): return Atom('Equals', x, y)
# This predicate is used to assert that two objects are the same.
# In particular, Equals(x,x) = True and Equals(x,y) = False iff x is not equal to y. 
# It can also be solved in other ways, without the Equals predicate!

def liar() -> Tuple[List[Formula], Formula]:
    def TellTruth(x): return Atom('TellTruth', x)
    def CrashedServer(x): return Atom('CrashedServer', x)
    john = Constant('john')
    susan = Constant('susan')
    nicole = Constant('nicole')
    mark = Constant('mark')
    formulas = []

    # We provide the formula for fact 0 here.
    formulas.append(Equiv(TellTruth(john), Not(CrashedServer(john))))
    
    # You should add 5 formulas, one for each of facts 1-5.
    # YOUR CODE HERE
    # Fact 1
    formulas.append(Equiv(TellTruth(susan), CrashedServer(nicole)))
    
    # Fact 2
    formulas.append(Equiv(TellTruth(mark), CrashedServer(susan)))
    
    # Fact 3
    formulas.append(Equiv(TellTruth(nicole), Not(TellTruth(susan))))
    
    # Fact 4
    formulas.append(Exists('$x', And(TellTruth('$x'), Forall('$y', Or(Equals('$x', '$y'), Not(TellTruth('$y')))))))
    
    # Fact 5
    formulas.append(Exists('$x', And(CrashedServer('$x'), Forall('$y', Or(Equals('$x', '$y'), Not(CrashedServer('$y')))))))
    
    # Query: Who crashed the server?
    query = CrashedServer('$x')
    return (formulas, query)

In [17]:
predForms, predQuery = liar()
assert len(predForms) == 6

In [18]:
predForms, predQuery = liar()
formula_id = 0
assert checkFormula('3a-' + str(formula_id), predForms[formula_id]) == True

You matched the 2 models
Example model: {'CrashedServer(john)'}


In [19]:
predForms, predQuery = liar()
formula_id = 1
assert checkFormula('3a-' + str(formula_id), predForms[formula_id]) == True

You matched the 2 models
Example model: set()


In [20]:
predForms, predQuery = liar()
formula_id = 2
assert checkFormula('3a-' + str(formula_id), predForms[formula_id]) == True

You matched the 2 models
Example model: set()


In [21]:
predForms, predQuery = liar()
formula_id = 3
assert checkFormula('3a-' + str(formula_id), predForms[formula_id]) == True

You matched the 2 models
Example model: {'TellTruth(nicole)'}


In [22]:
predForms, predQuery = liar()
formula_id = 4
assert checkFormula('3a-' + str(formula_id), predForms[formula_id]) == True

You matched the 4 models
Example model: {'TellTruth(mark)'}


In [23]:
predForms, predQuery = liar()
formula_id = 5
assert checkFormula('3a-' + str(formula_id), predForms[formula_id]) == True

You matched the 4 models
Example model: {'CrashedServer(john)'}


In [24]:
predForms, predQuery = liar()
assert checkFormula('3a-all', AndList(predForms)) == True

You matched the 1 models
Example model: {'TellTruth(nicole)', 'CrashedServer(john)'}


Please run the cell below to see if you have solved the problem :)

In [25]:
# Run this cell to solve the puzzle and find out who was the liar!
predForms, predQuery = liar()
kb = createModelCheckingKB()

filename = os.path.join('models', '3a-all.pklz')
objects, targetModels = pickle.load(gzip.open(filename))
for obj in objects:
    kb.tell(Atom('Object', obj))

for predForm in predForms:
    response = kb.tell(predForm)

response = kb.ask(predQuery)
showKBResponse(response)

Yes: ['john']
Maybe: []
No: ['nicole', 'susan', 'mark']


## 4. Inference (5 pts)

**You will designed this part by your own python scripts. Please do not write to this notebook.**

Given a Knowledge Base **(KB)** and a statement $\alpha$, both represented by prepositional logic and change to CNF. Determine if KB entails $\alpha$ (KB ⊨ $\alpha$) by resolution.

- Your report have at least 5 test cases for demo (test cases should not be too simple). 
- Write short evaluations about the advantages and disadvantages of resolution method for prepositional logic, propose your own solution for specific problem.

**a. Desribe the experiment data.**
- **Input data**: **KB** and $\alpha$ with CNF stored in **input.txt**. This file have the following structure:
    - The first line contains $\alpha$ statement.
    - The second line has N – the number of clauses in the KB.
    - N following lines are clauses in KB, one line for each clause.
    
Positive literal is represented by an uppercase character (A-Z). Negative literal is positive literal with minus (‘-‘) before the character.

Literals are linked by OR. Between literals and keywords there can be more than one space.

- **Output data**: The set of statements that are generated during the whole resolution process and the conclusion clause is stored in **output.txt**. This file must have the following structure:
    - The first line contains M1 – the number of generated clauses in first loop. M1 following lines describe those clauses (including empty clause), each line represents a clause. The empty clause is represented by “{}”
    - Next loops are also represented as ($M_2$, $M_3$,…, $M_n$ clauses) the first loop. 
    - The last line is the conclusion clause, which mean if “KB entails $\alpha$?”. Print YES if KB entails $\alpha$. NO for the opposite.
    - Remove redundant clause (same clauses in same loop, or the loops before).
    
    
- main function must:
    - Read the input data and store it in a suitable data structure.
    - Call PL-Resolution function for execute resolution algorithm.
    - Export the output data with the required format.


- Store the semantic information of true and false value in PL-RESOLUTION. Do not forget that we need to negate $\alpha$.

- <font color='red'>Literals in a clause is sorted by alphabetical ordering. </font>
- <font color='red'>Infered condition is checked in each loop as when all the new clauses are generated from the current KB.</font>
- <font color='red'>Clauses that have A ∨ B ∨ -B format and True value are similar to A ∨ True can be removed. </font>


Example: A KB and $\alpha$ clause in **input.txt**

| Input.txt | Output.txt | Notes |
| --- | --- | --- |
| -A | 3 |  |
| 4 | -A | (-A OR B) resolved with (-B)|
| -A OR B | B | (-A OR B) resolved with (negative of -A) |
| B OR -C | -C | (-C OR B) resolved with (-B) |
| A OR -B OR C | 4 |  |
| -B | -B OR C | (A OR C OR -B) resolved with (-A) |
|  | A OR C | (A OR C OR -B) resolved with (B) |
|  | A OR -B| (A OR C OR -B) resolved with (-C) |
|  | {}| (-B) resolved with (B) |
|  | **YES** | KB entails $\alpha$ because an empty clause exists in KB. |


Another example KB with a different $\alpha$

| Input.txt | Output.txt | Notes |
| --- | --- | --- |
| A | 2 | KB does NOT entail $\alpha$ because no new clause is generated and no empty clause is found.|
| 4 | -C |  |
| -A OR B | -B OR C |  |
| -C OR B | 2 |  |
| A OR C OR -B | A OR -B |  |
| -B | -A OR C |  |
|  | 1 |  |
|  | A OR -C |  |
|  | **0** |  |
|  | NO |  |







**b. Evaluations**

| **No.** | **Description** | **Ratio** |
| --- | --- | --- |
| 1 | Read input data and store in suitable data structure | 0.5 |
| 2 | Implementation of resolution method  | 1.0 |
| 3 | Inference process and results | 2.5 |
| 5 | Testcases, report, evaluations,  | 1.0 |

The end.